# you.com News API <> Llama Index

Creating a chatbot that can provide personalized news on any topic using you.com news API involves several steps, including obtaining the necessary credentials, and programming your chatbot to use the API. 

## Install required packages

In [ ]:
%%capture
! pip install llama-index-retrievers-you==0.1.2
! pip install dotenv==0.0.5

## Load API keys

In [1]:
# assumes a .env file exists with api keys YOU_API_KEY and OPENAI_API_KEY

from dotenv import load_dotenv
import os

load_dotenv()

True

## Create a retriever

In [ ]:
from llama_index.retrievers.you import YouNewsRetriever

you_api_key = os.environ["YOU_API_KEY"]

retriever = YouNewsRetriever(api_key=you_api_key)
retrieved_results = retriever.retrieve("national parks in the US")

print(retrieved_results[0].get_content())

In [3]:
# TODO until PR is merged, create it manually: https://github.com/run-llama/llama_index/pull/13934

from typing import List, Optional
from llama_index.core.schema import NodeWithScore, TextNode
import requests


class YouNewsRetriever():
    """You retriever."""

    def __init__(
        self,
        api_key: Optional[str] = None
    ) -> None:
        """Init params."""
        self._api_key = api_key or os.environ["YOU_API_KEY"]

    def retrieve(self, query_bundle: str) -> List[NodeWithScore]:
        """Retrieve."""
        headers = {"X-API-Key": self._api_key}
        results = requests.get(
            f"https://api.ydc-index.io/news?q={query_bundle}",
            headers=headers,
        ).json()

        news_hits = [f'{news["description"]}\nURL: {news["url"]}\nage: {news["age"]}' for news in results["news"]["results"]]
        return [NodeWithScore(node=TextNode(text=s), score=1.0) for s in news_hits]

you_api_key = os.environ["YOU_API_KEY"]

retriever = YouNewsRetriever(api_key=you_api_key)
retrieved_results = retriever.retrieve("national parks in the US")

print(retrieved_results[0].get_content())

Our 63 US National Parks across America are a collection of some of the most pristine and beautifully preserved landscapes in the world. From sea to shining sea, nationally protected treasures are filled with stunning views, remarkable wildlife, and historic landmarks.
URL: https://brightstandards.com/us-national-parks-list-map/
age: 5 days ago


# Create a chat engine

Conceptually, it is a stateful analogy of a Query Engine. 

In [4]:
from llama_index.core.chat_engine.context import ContextChatEngine

chat_engine = ContextChatEngine.from_defaults(retriever=retriever)
response = chat_engine.chat("Canada")

print(response)

The recent news articles provide a snapshot of various events and developments in Canada. Here are some key highlights:

1. The Bank of Canada cut its key policy rate, becoming the first G7 country to do so in four years. Economists predict further cuts in July.
   Source: Reuters, Yahoo Finance, CNBC

2. Canada Border Services Agency workers are set to strike, potentially causing border chaos.
   Source: CBC News, WCAX

3. Canada demands a portion of revenue from streaming services to support local news and content.
   Source: Ars Technica, Reuters

4. Wall Street anticipates a weaker Canadian dollar and more interest rate cuts following the Bank of Canada's move.
   Source: Bloomberg

5. There are calls to end the practice of "flagpoling" in Canada.
   Source: WCAX

6. Canadian officials are preparing for a meeting with French President Macron, US President Biden, UK's King Charles III, and Canada's PM Trudeau to commemorate D-Day.
   Source: The Guardian

7. Online streaming service

### Include a timeframe

In [5]:
from llama_index.core.chat_engine.context import ContextChatEngine

chat_engine = ContextChatEngine.from_defaults(retriever=retriever)
response = chat_engine.chat("Norway in Jan 2024")

print(response)

In January 2024, Norway experienced some significant events and developments:

1. Norway's $1.6 trillion sovereign wealth fund, the world's largest, reported a record profit of 2.22 trillion crowns ($213 billion) in 2023, driven by strong returns on its investments in technology stocks.

2. January saw combined electric vehicles (EVs) at a 93.9% share in Norway, with a record 92.1% being full electrics (BEVs) and 1.8% plug-in hybrids (PHEVs). This marked a significant increase compared to the previous year.

3. Norway experienced a market crash of -76.6% in January 2023, leading to the lowest market performance in 61 years due to price increases.

4. The country saw a surge in new electric car registrations, with 4,717 new electric cars and 94 plug-in hybrids registered in January 2024.

5. Norway's internet penetration stood at 99.0%, with 5.44 million internet users and 4.49 million social media users in January 2024.

These events showcase Norway's strong focus on sustainability, te

# Create a chat engine with memory

By keeping track of the conversation history, it can answer questions with past context in mind.
 
`condense_plus_context` - A combination of condense_question and context. Look at the chat history and re-write the user message to be a retrieval query for the index. The retrieved text is inserted into the system prompt, so that the chat engine can either respond naturally or use the context from the query engine.

In [6]:
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.chat_engine import CondensePlusContextChatEngine

memory = ChatMemoryBuffer.from_defaults(token_limit=1500)

chat_engine = CondensePlusContextChatEngine.from_defaults(
    retriever=retriever,
    memory=memory,
    verbose=True,
)

In [63]:
response = chat_engine.chat("Salesforce")
print(response)

Condensed question: Salesforce
Context: The cloud software giant just disappointed a lot of investors.
URL: https://www.fool.com/investing/2024/06/03/is-salesforce-stock-a-buy-now/
age: 2 days ago

Insider buying and selling is tracked closely by many professional fund managers.
URL: https://www.cnbc.com/2024/06/03/ceos-of-general-motors-salesforce-among-corporate-insiders-selling-stock-last-month.html
age: 1 day ago

Here's how Bret Jensen, usually not a big tech investor, is taking advantage of Saleforce's big selloff.
URL: https://pro.thestreet.com/trade-ideas/getting-salesforce-on-the-cheap
age: 3 days ago

The sharp drop in the CRM stock price came after the company declared its first quarter (FY 2025) earnings. It missed the revenue consensus..
URL: https://www.forbes.com/sites/greatspeculations/2024/06/03/salesforce-stock-dropped-more-than-20-in-one-day-whats-next/
age: 1 day ago

But the shocking Salesforce quarter sent me off to dig into the tech-heavy Nasdaq, specifically the

In [64]:
# uses memory to query related things
response = chat_engine.chat("Hubspot")
print(response)

Condensed question: What specific details or questions do you have about Hubspot?
Context: Find answers and general information quickly about the ads tool in HubSpot.
URL: https://knowledge.hubspot.com/ads/ads-faq
age: 1 month ago

CRM & Sales Hub Ask and answer questions about using HubSpot’s CRM and Sales Hub. Marketing Hub Discuss and learn HubSpot’s marketing tools and inbound strategy. Service Hub Learn about Service Hub and share your expertise.
URL: https://knowledge.hubspot.com/integrations/schedule-a-meeting-with-a-contact-in-a-record
age: May 2, 2024

Find answers to common questions about the traffic analytics tool in HubSpot.
URL: https://knowledge.hubspot.com/reports/traffic-analytics-faq
age: May 1, 2024

When a buyer makes a payment or subscription is created, HubSpot stores details about the payment or subscription in specific properties.
URL: https://knowledge.hubspot.com/payments/hubspots-payments-and-subscriptions-properties
age: April 30, 2024

Create a chatflow wit